In [1]:
import json
import pandas
import os
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from  itertools import tee, chain, islice
from math import log
from operator import itemgetter
from itertools import takewhile
import requests

In [2]:
root_dir = '/home/marcin/Desktop/SemestrVIII/PJN'
year = "2018"
json_data_dir = f"{root_dir}/data/json"


def extract_file_number(filename):
    return int(filename.split('-')[1].split('.')[0])

def is_2017_in_file(filename):
    return 2716 <= extract_file_number(filename) <= 3163 

def is_2018_in_file(filename):
    return 3163 <= extract_file_number(filename) <=  3173

In [3]:
def window(it, size=2):
    yield from zip(*[islice(it, s, None) for s, it in enumerate(tee(it, size))])

In [4]:
def clean_text(line):
    _id, text = line
    notags = re.sub(r"<[^>]*>", " ", text)
    nobreaks =  re.sub(r"-\n", " ", notags)
    nodigits =  re.sub(r"\d+", " ", nobreaks)
    return _id, re.sub(r"\b[XVILMC]+\b", "", nodigits)

In [5]:
def judgement_texts(filename):
    with open(os.path.join(json_data_dir, filename), 'r') as jsonFile:
        judgements = json.load(jsonFile)['items'] 
    year_filtered = filter(lambda item: year in item['judgmentDate'], judgements)
    yield from map(lambda item: (item['id'],item['textContent']), year_filtered)

In [6]:
def process_response(line):
    word, tags = line.split('\t')[1:3]
    grammatical_class = tags.split(':')[0]
    return "{}:{}".format(word.lower().replace(' ','_'), grammatical_class)

In [7]:
def tag_text(judgement):
    _id,text = judgement 
    r = requests.post("http://localhost:9200", data=text.encode('utf-8'))
    lines= r.content.decode('utf-8').split('\n') 
    relevant = filter(lambda line: line.startswith('\t'),lines)
    tagged =  map(process_response, relevant)
    words =  filter(lambda x: re.match("\w+",x[0]),tagged) 
    return _id, words

In [8]:
def write_tagging(tagging,fileout):
   with open(fileout,'w') as file:
        for _id, tags in tagging:
            file.write("{}; ".format(_id))
            file.write(' '.join(x.replace(' ','_') for x in tags))
            file.write(' '.join(x for x in tags))
            file.write("\n")
            

In [9]:
json_files = os.listdir(json_data_dir)
judgements_files= filter(is_2018_in_file, json_files)
texts =  chain.from_iterable(map( judgement_texts, judgements_files))
cleaned_text = map(clean_text,texts)

In [ ]:
tagging_unigrams = map(tag_text, cleaned_text)
write_tagging(tagging_unigrams,'tagging-2017')